# Installation, Converting and Registering Image Classification Model by ModelCI

This is a getting started tutorial for those who are new to ML-ModelCI, by the end of this tutorial, you will be able to: 

- Install ML-ModelCI.
- Setup environment.
- Start and stop ModelCI service.
- Master basic usages of ML-ModelCI, such as model loading, registering,retrieving and converting.
- Have a basic understanding of machine learning model lifecycle.

## 1. Installation

Here are some prequisities before installation

- Python version: 3.7 or higher
- Docker service installed and started
- Manually install [TensorRT](https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html) if your linux distribution is not Ubuntu

We can install the ModelCI python package based on <https://github.com/cap-ntu/ML-Model-CI/#installation-using--pip>.

In [1]:
%%bash
git clone https://github.com/cap-ntu/ML-Model-CI.git
cd ML-Model-CI
pip install -q .

## 2. Start the ModelCI Service

For the first time you start the ModelCI service, you need to setup all the environment variables in a single `.env` file by the following script:

In [2]:
# set environment variables 
!python scripts/generate_env.py

Read env-backend.env ...
Read env-mongodb.env ...
Read env-frontend.env ...
Write .env for backend with setup:
 {
  "PROJECT_NAME": "modelci",
  "SERVER_HOST": "localhost",
  "SERVER_PORT": "8000",
  "SECRET_KEY": "2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19",
  "MONGO_HOST": "localhost",
  "MONGO_PORT": "27017",
  "MONGO_USERNAME": "modelci",
  "MONGO_PASSWORD": "modelci@2020",
  "MONGO_DB": "modelci",
  "MONGO_AUTH_SOURCE": "modelci",
  "BACKEND_CORS_ORIGINS": "localhost:3333"
}
Write .env for frontend with setup:
 {
  "HOST": "localhost",
  "PORT": "3333",
  "REACT_APP_BACKEND_URL": "localhost:8000"
}


Then start the modelci service by following command:

In [3]:
!modelci service init

2021-04-28 00:58:48.608554: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-04-28 00:58:53,839 - ml-modelci Docker Container Manager - INFO - Container name=mongo-49205 stared
2021-04-28 00:58:55,196 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-12065 started.
2021-04-28 00:58:56,522 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-5717 started.
2021-04-28 00:58:57,853 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-31125 stared
2021-04-28 00:58:58,377 - modelci backend - INFO - Uvicorn server listening on http://localhost:8000, check full log at /home/shanshan/tmp/modelci.log


## 2. Register ResNet50 Model


Firstly, we load pre-trained resnet50 model from torchvision, you can refer to <https://pytorch.org/docs/stable/torchvision/models.html> for more examples of pretrained models.

In [4]:
from torchvision import models
model = models.resnet50(pretrained=True)

Then we specify five basic attributes of this model:
- architecture
- framework
- engine
- task
- version

and save the model in the generated path

In [5]:
import os
import torch
from pathlib import Path
from modelci.types.models.common import Engine, Task, Framework, Metric, ModelStatus, IOShape, DataType
from modelci.types.models import MLModel
from modelci.hub.utils import generate_path_plain

In [6]:
architecture = 'ResNet50'
framework = Framework.PyTorch
engine = Engine.PYTORCH
version = 1
task = Task.Image_Classification
model_save_path = generate_path_plain(architecture, task, framework, engine, version)

if not Path.is_dir(Path(model_save_path).parent):
    os.makedirs(Path(model_save_path).parent, exist_ok=True)
torch.save(model, model_save_path)

Then we can build a MLModel instance and register it into modelhub

In [7]:
mlmodel = MLModel(
    weight=Path(model_save_path),
    architecture=architecture,
    dataset='ImageNet',
    framework=framework,
    engine=engine,
    version=version,
    metric={Metric.acc: 0.80},
    task=task,
    inputs=[IOShape(name="input", shape=[-1, 3, 224, 224], dtype=DataType.TYPE_FP32)],
    outputs=[IOShape(name="output", shape=[-1, 1000], dtype=DataType.TYPE_FP32)],
    model_status=[ModelStatus.PUBLISHED]
)

In [8]:
from modelci.hub.manager import register_model
register_model(mlmodel, convert=True, profile=False)

2021-04-28 14:30:03,066 - converter - INFO - Torchscript format converted successfully
2021-04-28 14:30:03,066 - converter - INFO - Torchscript format converted successfully
2021-04-28 14:30:03,066 - converter - INFO - Torchscript format converted successfully
2021-04-28 14:30:03,066 - converter - INFO - Torchscript format converted successfully
2021-04-28 14:30:05,669 - converter - INFO - ONNX format converted successfully
2021-04-28 14:30:05,669 - converter - INFO - ONNX format converted successfully
2021-04-28 14:30:05,669 - converter - INFO - ONNX format converted successfully
2021-04-28 14:30:05,669 - converter - INFO - ONNX format converted successfully


[MLModel(architecture='ResNet50', framework=<Framework.PyTorch: 1>, engine=<Engine.PYTORCH: 7>, version=1, dataset='ImageNet', metric={<Metric.acc: 0>: 0.8}, task=<Task.Image_Classification: 0>, inputs=[IOShape(shape=[-1, 3, 224, 224], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)], outputs=[IOShape(shape=[-1, 1000], dtype=<DataType.TYPE_FP32: 11>, name='output', format=<ModelInputFormat.FORMAT_NONE: 0>)], id=ObjectId('608900e967de5e180caf93ec'), parent_model_id=None, weight=Weight(__root__=ObjectId('608900e867de5e180caf9262')), profile_result=None, status=<Status.Unknown: 0>, model_input=None, model_status=[<ModelStatus.PUBLISHED: 0>], creator='shanshan', create_time=datetime.datetime(2021, 4, 28, 6, 29, 56, 12289)),
 MLModel(architecture='ResNet50', framework=<Framework.PyTorch: 1>, engine=<Engine.TORCHSCRIPT: 2>, version=1, dataset='ImageNet', metric={<Metric.acc: 0>: 0.8}, task=<Task.Image_Classification: 0>, inputs=[IOShape(shape=[-1, 3, 2

## 3. Retrieve Models
By default, Converter will automatically convert registered models into optimized formats,PyTorch model can be converted to TorchScipt and ONNX formats, so we can retrieve two models from ModelHub.

In [10]:
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
    architecture='ResNet50',
    framework=Framework.PyTorch,
    version=1
)

In [11]:
len(retrieved_models)

3

We can compare detatiled information of these three models

In [13]:
retrieved_models[0].__dict__

{'architecture': 'ResNet50',
 'framework': <Framework.PyTorch: 1>,
 'engine': <Engine.PYTORCH: 7>,
 'version': 1,
 'dataset': 'ImageNet',
 'metric': {<Metric.acc: 0>: 0.8},
 'task': <Task.Image_Classification: 0>,
 'inputs': [IOShape(shape=[-1, 3, 224, 224], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'outputs': [IOShape(shape=[-1, 1000], dtype=<DataType.TYPE_FP32: 11>, name='output', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'id': ObjectId('608900e967de5e180caf93ec'),
 'parent_model_id': None,
 'weight': Weight(__root__=ObjectId('608900e867de5e180caf9262')),
 'profile_result': None,
 'status': <Status.Unknown: 0>,
 'model_input': None,
 'model_status': [<ModelStatus.PUBLISHED: 0>],
 'creator': 'shanshan',
 'create_time': datetime.datetime(2021, 4, 28, 6, 29, 56, 12000, tzinfo=<bson.tz_util.FixedOffset object at 0x7f7b8b5abfd0>)}

In [15]:
retrieved_models[1].__dict__

{'architecture': 'ResNet50',
 'framework': <Framework.PyTorch: 1>,
 'engine': <Engine.TORCHSCRIPT: 2>,
 'version': 1,
 'dataset': 'ImageNet',
 'metric': {<Metric.acc: 0>: 0.8},
 'task': <Task.Image_Classification: 0>,
 'inputs': [IOShape(shape=[-1, 3, 224, 224], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'outputs': [IOShape(shape=[-1, 1000], dtype=<DataType.TYPE_FP32: 11>, name='output', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'id': ObjectId('608900ee67de5e180caf9577'),
 'parent_model_id': None,
 'weight': Weight(__root__=ObjectId('608900ed67de5e180caf93ed')),
 'profile_result': None,
 'status': <Status.Unknown: 0>,
 'model_input': None,
 'model_status': [<ModelStatus.CONVERTED: 1>],
 'creator': 'shanshan',
 'create_time': datetime.datetime(2021, 4, 28, 6, 29, 56, 12000, tzinfo=<bson.tz_util.FixedOffset object at 0x7f7b8b5abfd0>)}

In [17]:
retrieved_models[2].__dict__

{'architecture': 'ResNet50',
 'framework': <Framework.PyTorch: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': 1,
 'dataset': 'ImageNet',
 'metric': {<Metric.acc: 0>: 0.8},
 'task': <Task.Image_Classification: 0>,
 'inputs': [IOShape(shape=[-1, 3, 224, 224], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'outputs': [IOShape(shape=[-1, 1000], dtype=<DataType.TYPE_FP32: 11>, name='output', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'id': ObjectId('608900f067de5e180caf9702'),
 'parent_model_id': None,
 'weight': Weight(__root__=ObjectId('608900ee67de5e180caf9578')),
 'profile_result': None,
 'status': <Status.Unknown: 0>,
 'model_input': None,
 'model_status': [<ModelStatus.CONVERTED: 1>],
 'creator': 'shanshan',
 'create_time': datetime.datetime(2021, 4, 28, 6, 29, 56, 12000, tzinfo=<bson.tz_util.FixedOffset object at 0x7f7b8b5abfd0>)}

## 4. Convert Models
We can convert models mannually. 

You can refer to <https://github.com/cap-ntu/ML-Model-CI/blob/master/docs/tutorial/convert.md> for more details.

In the following example, we will convert ONNX model into TensorRT format.

In [19]:
# get ONNX model saved path
onnx_path = retrieved_models[2].saved_path
onnx_path

PosixPath('/home/shanshan/.modelci/ResNet50/pytorch-onnx/image_classification/1.onnx')

In [21]:
# set TensorRT format model save path
architecture = 'ResNet50'
framework = Framework.PyTorch
version = 1
task = Task.Image_Classification
trt_model_save_path = generate_path_plain(architecture, task, framework, Engine.TRT, version)
trt_model_save_path

PosixPath('/home/shanshan/.modelci/ResNet50/pytorch-trt/image_classification/1')

In [23]:
from modelci.hub.converter import convert
convert(
    model=retrieved_models[2].saved_path,
    src_framework='onnx',
    dst_framework='trt',
    save_path=trt_model_save_path,
    inputs=retrieved_models[2].inputs,
    outputs=retrieved_models[2].outputs
)

Loading ONNX file from path /home/shanshan/.modelci/ResNet50/pytorch-onnx/image_classification/1.onnx...


True

## 5.Stop the ModelCI Service

In [25]:
!conda activate test && modelci service stop

2021-04-28 00:59:10.251530: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-04-28 00:59:12,892 - ml-modelci Docker Container Manager - INFO - Container name=gpu-metrics-exporter-31125 stopped.
2021-04-28 00:59:13,535 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-12065 stopped.
2021-04-28 00:59:14,859 - ml-modelci Docker Container Manager - INFO - Container name=mongo-49205 stopped.
2021-04-28 00:59:14,994 - modelci backend - WARNING - No process is listening on port 8000


In [26]:
!conda activate test && modelci service clean

2021-04-28 00:58:27.911203: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-04-28 00:58:30,634 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-53037 stopped.
2021-04-28 00:58:31,639 - ml-modelci Docker Container Manager - INFO - Container name=mongo-50811 stopped.
2021-04-28 00:58:31,742 - modelci backend - WARNING - No process is listening on port 8000
2021-04-28 00:58:31,796 - ml-modelci Docker Container Manager - INFO - Container a81962bf3b3d89e55a3588831b050cfd43639dc94a524402737a64982f428215 is removed.
2021-04-28 00:58:31,847 - ml-modelci Docker Container Manager - INFO - Container 5d9576a66007d8434400fbaf1b2a0f710f3896e21704d30bfd4d1d8694285a8a is removed.
2021-04-28 00:58:31,916 - ml-modelci Docker Container Manager - INFO - Container c4eabfb515aaafcd3497ae44ff59172714aa9bd58b92cbfd943aea5c4e44d542 is removed.
2021-04-28 00:58:31,986 - ml-modelci Docker Container Manager - INFO - C

## License

```raw
Copyright 2020 Nanyang Technological University, Singapore

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```